::::{margin}
:::{card} Create a virtual particle that represents the state pγ in `QRules`
Report-0
^^^
+++
✅&nbsp;[ComPWA/gluex-nstar#5](https://github.com/ComPWA/gluex-nstar/issues/5)
:::
::::

# Virtual particle that represents the state pγ 

Address to issue [ComPWA/gluex-nstar#5](https://github.com/ComPWA/gluex-nstar/issues/5), this section is an investigation of creating a virtual particle that represents the state $p \gamma$

As an example of side note, create a virtual particle that represents state $e^+ e^-$ can be seen [here](https://qrules.readthedocs.io/0.10.x/usage/particle/#adding-custom-particle-definitions-through-python).

`QRules` is used for this task

In [ ]:
from qrules.particle import load_pdg

particle_db = load_pdg()
print("Number of loaded particles:", len(particle_db))

In [ ]:
particle_db.find(2212)

In [ ]:
particle_db.find("p")

In [ ]:
particle_db.find(22)

In [ ]:
particle_db.find("gamma")

In [ ]:
from IPython.display import Math

sigmas = particle_db.filter(
    lambda p: p.name.startswith("p") and p.mass == 0.93827208816 and p.charge == 1
)
Math(", ".join(p.latex for p in sigmas))

In [ ]:
from IPython.display import Math

sigmas = particle_db.filter(lambda p: p.name.startswith("gamma"))
Math(", ".join(p.latex for p in sigmas))

In [ ]:
proton = particle_db["p"]
proton

In [ ]:
gamma = particle_db["gamma"]
gamma

In [ ]:
from qrules.particle import Particle, Spin

pgamma = Particle(
    name="p",
    latex=r"p \gamma",
    spin=0.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99999999,
)
pgamma

In [ ]:
pgamma_s3over2 = Particle(
    name="pgamma",
    latex=R"p \gamma (s3/2)",
    spin=1.5,
    mass=4.101931071854584,
    charge=1,
    isospin=Spin(1 / 2, +1 / 2),
    baryon_number=1,
    parity=-1,
    pid=99999999,
)
pgamma_s3over2

In [ ]:
pgamma_s3over2.latex

In [ ]:
pgamma.latex

The `pgamma` virtual state generated via `QRules` will be used in later stages!